In [2]:
import requests
import json
import openai
  
openai.api_key = open(".tokens/openai", "r").read()
print("Started")

Started


In [3]:
# Uses anki-connect to connect to anki
response = requests.post('http://localhost:8765', data='{"action": "deckNames", "version": 6}', headers={'Content-type': 'application/json'})
print(response.json())

{'result': ['A2_Wortliste_Goethe', 'B1_Wortliste_DTZ_Goethe', 'Deutsch B1', 'Deutsch B1+', 'DW Nicos Weg B1', 'Standard'], 'error': None}


In [4]:
def invoke(method, **params):
    request_data = json.dumps({
        'action': method,
        'params': params,
        'version': 6
    })

    response = requests.post('http://localhost:8765', data=request_data)
    response_data = json.loads(response.text)

    if response_data.get('error'):
        raise Exception(f"Error: {response_data['error']}")
    else:
        return response_data['result']

In [48]:
# prop:ivl<=20 means that they should have less than 20 days of interval
# rated:1:2 -> cards answered hard today
# rated:1 -> cards rated today
# rated:7 -> cards rated in the last 7 days
# introduced:1 -> cards introduced today
reviewed_card_ids = invoke('findCards', query=f'rated:1:2')
cards = invoke('cardsInfo', cards=reviewed_card_ids)
print(len(cards))

51


In [49]:
# Unsupported yet
# invoke('getReviewsOfCards', cards=['1497484862085'])

In [51]:
comma_separated_repeated_words = ', '.join({card['fields']['Wort_DE']['value'] for card in cards})
task = "Create a short story in German for level B1 that helps you read the following words: " + comma_separated_repeated_words
print(task)

Create a short story in German for level B1 that helps you read the following words: Qualität, Reise, nehmen, Messe, Bahnsteig, Quiz, Poster, zuverlässig, Fahrplan, oft, mitnehmen, reden, Parfüm, abonnieren, notwendig, neblig, aussteigen, Terasse, passen, Papier, nichts, Platz, Problem, Paar, dorther, Kunde, Reis, Park, Preis, Prospekt, hinein, Vorort, knapp, plötzlich, Nachbar, Produkt, nützlich, Radio, preiswert, Reifen, Praktikum, Nummer, Ort, abnehmen, Museum, nett


In [43]:
result = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a German Tutor."},
        {"role": "user", "content": task}
    ]
)
content = result.to_dict_recursive()['choices'][0]["message"]["content"]

In [46]:
test_result = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a German Tutor, generate a test for B1 language level for the following test."},
        {"role": "user", "content": content}
    ]
)
test_content = test_result.to_dict_recursive()['choices'][0]["message"]["content"]

In [47]:
# save to file with the current datetime
import datetime
with open(f"german-text-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.md", "w") as f:
    f.write("## Task:\n\n")
    f.write(task)
    f.write("\n\n## Story:\n\n")
    f.write(content)
    f.write("\n\n## Test\n")
    f.write(test_content)
